<a href="https://colab.research.google.com/github/axelDMC/DataScience/blob/main/WhatsApp_Chat_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the required libraries

In [1]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=fd45ee7512bbe73568ca521a0c5f477f3abd3ba936112c5165626d82cc0f23de
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [2]:
import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


Definition of Functions

In [3]:
# Extract the Date time
def date_time(s):
    # pattern='^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    pattern='^([0-9]+)(/)([0-9]+)(/)([0-9]+) ([0-9]+):([0-9]+) -'
           #'^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]) ([0-9]+):([0-9]+) - Desbloqueaste a este contacto$'
    result = re.match(pattern, s)
    if result:
      return True
    return False

# Extract contacts
def find_contact(s):
  s=s.split(":")
  if len(s)==2:
    return True
  else:
    return False

# Extract message
def getMessage(line):
  splitline = line.split(' - ')
  datetime = splitline[0];
  date, time = datetime.split(' ')
  message = " ".join(splitline[1:])

  if find_contact(message):
    splitmessage=message.split(": ")
    author=splitmessage[0]
    message=" ".join(splitline[1:])
  else:
    author=None
  return date, time, author, message

In [4]:
data=[]
conversation='/content/drive/MyDrive/Chat de WhatsApp con Brayan Unsa.txt'
with open(conversation, encoding="utf-8") as fp:
  fp.readline()
  messageBuffer=[]
  date, time, author = None, None, None
  while True:
    line=fp.readline()
    if not line:
      break
    line=line.strip()
    # print(date_time(line), line)
    if date_time(line):
      if len(messageBuffer)>0:
        data.append([date, time, author, ''.join(messageBuffer)])
      messageBuffer.clear()
      date, time, author, message=getMessage(line)
      messageBuffer.append(message)
    else:
      messageBuffer.append(line)


# Analyse the sentiments

In [6]:
df=pd.DataFrame(data, columns=["Date", "Time", "Contact", "Message"])
df['Date']=pd.to_datetime(df['Date'])
data=df.dropna()
import nltk
nltk.download('vader_lexicon')
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# sid = SentimentIntensityAnalyzer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments=SentimentIntensityAnalyzer()
data["positive"]=[sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["negative"]=[sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["neutral"]=[sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

data.head(50)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Date,Time,Contact,Message,positive,negative,neutral
2,2022-04-06,10:33,Axel,Axel: Maricon,0.000,0.000,1.000
3,2022-04-06,10:33,Brayan Unsa,Brayan Unsa: Hols,0.000,0.000,1.000
4,2022-04-06,17:24,Brayan Unsa,Brayan Unsa: <Multimedia omitido>,0.000,0.000,1.000
5,2022-04-06,17:31,Axel,Axel: Jaja había que esperar,0.000,0.000,1.000
6,2022-04-06,17:31,Brayan Unsa,Brayan Unsa: 😪😪😪 tmt,0.000,0.000,1.000
7,2022-04-06,21:09,Axel,Axel: Que noticias cholo ?,0.000,0.000,1.000
8,2022-04-06,21:55,Brayan Unsa,Brayan Unsa: Tmr casi muero dos veces hoy,0.000,0.000,1.000
9,2022-05-06,4:56,Axel,Axel: Jajaja xqqq ?,0.000,0.000,1.000
10,2022-05-06,8:30,Brayan Unsa,Brayan Unsa: Oe,0.000,0.000,1.000
11,2022-05-06,8:30,Brayan Unsa,Brayan Unsa: Hoy a las 9?,0.000,0.000,1.000


In [8]:
x=sum(data["positive"])
y=sum(data["negative"])
z=sum(data["neutral"])

def score(a,b,c):
  if (a>b) and (a>c):
    print("Positive ")
  if (b>a) and (b>c):
    print("Negative")
  if (c>a) and (c>b):
    print("Neutral")

score(x,y,z)

Neutral
